In [ ]:
import pandas as pd
import pandas_utils as pu

import matplotlib.pyplot as plt
import matplotlib
import upsetplot as ups
import seaborn as sns

In [ ]:
library = "acquisition"
def save_fig(file_name):
  plt.rcParams["pdf.fonttype"] = 42
  plt.tight_layout()
  plt.savefig(r"C:\git\msn_library\figures/{}/{}.png".format(library, file_name), dpi=300, bbox_inches="tight", transparent=True)
  plt.savefig(r"C:\git\msn_library\figures/{}/{}.pdf".format(library, file_name), bbox_inches="tight", transparent=True)
  plt.savefig(r"C:\git\msn_library\figures/{}/{}.svg".format(library, file_name), bbox_inches="tight", transparent=True)
sns.set_theme(font_scale=1, style="white")

In [ ]:
file = r"C:\git\msn_library\data\acquisition_results\all_lib_no_filter.tsv"

In [ ]:
df = pu.read_dataframe(file)

In [ ]:
def combine_polarity(old, new):
  if old == "both":
    return "both"
  match new:
    case "both":
      return new
    case "positive":
      return "both" if old == "negative" else "positive" 
    case "negative":
      return "both" if old == "positive" else "negative" 
    case _:
      return old
  

unique_dict = {}
for inchikey, polarity in zip(df["inchikey"], df["polarity"]):
  oldpolarity = unique_dict.get(inchikey, "missing")
  unique_dict[inchikey] = combine_polarity(oldpolarity, polarity)
  
df["new_polarity"] = [unique_dict.get(inchikey) for inchikey in df["inchikey"]]
df = df.sort_values(by=["detected"]).drop_duplicates(["new_polarity", "inchikey"]).sort_index()
# df[df["inchikey"].duplicated(keep=False)][["inchikey", "polarity", "new_polarity"]]
df

In [ ]:
filtered = df[df["new_polarity"] != "missing"]
filtered.drop_duplicates(["split_inchikey"])

In [ ]:
df.drop_duplicates(["split_inchikey"])

In [ ]:
df["positive"] = (df["new_polarity"] == "positive") | (df["new_polarity"] == "both")
df["negative"] = (df["new_polarity"] == "negative") | (df["new_polarity"] == "both")
df["both"] = df["new_polarity"] == "both"
df["missing"] = df["new_polarity"] == "missing"


In [ ]:
file_name = "acquired_upset_plot"
acquired_combined = ups.from_indicators(["positive", "negative", "missing"], data=df)
ups.plot(acquired_combined)
save_fig(file_name)
matplotlib.pyplot.show()